In [ ]:
! pip install rarfile

In [ ]:
import numpy as np

In [ ]:
import json
import os
import rarfile
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1. Data Preprocessing

## A. text dataset




In [ ]:
# Define the path to the JSON file
json_file_path = '/content/drive/MyDrive/nytimes_train.json'

# Read the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)
#  first few article samples
num_samples_to_print = 5
for i, article in enumerate(data[:num_samples_to_print]):
    print(f"Article {i+1}: {article}")


len(data)
print(type(data))

Article 1: {'section': 'Theater', 'headline': "Before 'Moonlight' and 'The Walking Dead,' a Friendship Born in the Classroom", 'article_url': 'https://www.nytimes.com/2017/02/21/theater/danai-gurira-andre-holland-walking-dead.html', 'article': 'Danai Gurira and Andre Holland in a theater at New York University, where they met in the Tisch Graduate Acting Program.\n\nBehind every successful person are relationships that helped forge a path. But the stories of these friendships, collaborations, alliances, romances or rivalries often are lost in the glow of achievement. In this new feature, we explore a personal connection that made a difference in the lives of two artists. Andre Holland never thought much about writing his own monologues when he attended the Tisch graduate acting program at New York University. But one day, early in his first semester in 2003, he watched another African-American student, Danai Gurira, forgo the usual speeches by white characters and perform her own mater

In [ ]:
# Define lists to store extracted features
headlines = []
abstracts = []
captions = []

# Iterate over each article in the dataset
for article in data:
    # Extract headline, abstract, and caption
    headline = article['headline']
    abstract = article['abstract']
    caption = article['caption']

    # Append extracted features to respective lists
    headlines.append(headline)
    abstracts.append(abstract)
    captions.append(caption)

# Display the extracted features
for i in range(len(headlines)):
    print(f"Article {i+1}:\nHeadline: {headlines[i]}\nAbstract: {abstracts[i]}\nCaption: {captions[i]}\n")


Article 1:
Headline: Before 'Moonlight' and 'The Walking Dead,' a Friendship Born in the Classroom
Abstract: André Holland first saw Danai Gurira at New York University, performing a monologue she wrote. Since then, they have acted and written together.
Caption: Danai Gurira and André Holland in a theater at New York University, where they met in the Tisch Graduate Acting Program.

Article 2:
Headline: What's on TV Wednesday: 'Crip Camp' and 'Dark Phoenix'
Abstract: The documentary "Crip Camp: A Disability Revolution" debuts on Netflix. And "Dark Phoenix" airs on HBO.
Caption: Judy Heumann in &ldquo;Crip Camp: A Disability Revolution.&rdquo;

Article 3:
Headline: Rays Stick to Their Plan and Reach a 2nd World Series
Abstract: Guided by the club's strict principles, Manager Kevin Cash pulled his pitcher at a crucial point in Game 7 and was rewarded with a victory over the Houston Astros.
Caption: The Rays celebrated after the final out of their Game 7 win.

Article 4:
Headline: For Robe

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Article 16406:
Headline: Emilia Clarke: Daenerys's 'Game of Thrones' Turn 'Was a Huge Shock'
Abstract: In an interview, Clarke discussed the emotional backstory for her character's fate in the show's final episodes.
Caption: Emilia Clarke in the series finale of "Game of Thrones."

Article 16407:
Headline: 'Survivor' Removes Player Accused of Inappropriate Touching
Abstract: The contestant, Dan Spilo, had been warned about his behavior. He was removed after an unspecified off-camera incident.
Caption: A scene from this season's episode of "Survivor."

Article 16408:
Headline: Calvin Klein's Style Sharknado
Abstract: How to dress for the age of "Fear," courtesy of Raf Simons. And other options from Oscar de la Renta.
Caption: Calvin Klein, spring 2019.

Article 16409:
Headline: Ann Sarnoff, a Hollywood Outsider, Will Lead Warner Bros. Studio
Abstract: Ms. Sarnoff will succeed Kevin Tsujihara, who was ousted amid a sex scandal, as the leader of the storied studio behind "Casablanca" and

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.

Article 35685:
Headline: The Era of Small Government Is Over
Abstract: We're going to have to reach much deeper than stimulus and bailouts into the way we conduct business with each other.
Caption: So far, none of the proposed stimulus programs coming out of Washington are enough to stop a recession.

Article 35686:
Headline: Striking Down the Queen Won't Save You From the Swarm
Abstract: It's a common trope in science fiction, but hives in nature are not dependent on any central node for their function.
Caption: Captain Jean Luc-Picard, right, facing the Borg Queen in "Star Trek: First Contact."

Article 35687:
Headline: At New York City Ballet, Balancing the Art of Transition
Abstract: The company is in flux on many fronts, but Teresa Reichlen's speech at the gala set out a fresh agenda — and dancers are delivering peak performances.
Caption: Teresa Reichlen, left, and Abi Stafford in George Balanchine's "Concerto Barocco." In a spe

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:


# Combine all text data for tokenization
all_text = headlines + abstracts + captions

# Initialize a tokenizer
tokenizer = Tokenizer()

# Fit tokenizer on all text data to build vocabulary
tokenizer.fit_on_texts(all_text)

# Tokenize headlines, abstracts, and captions
headline_seqs = tokenizer.texts_to_sequences(headlines)
abstract_seqs = tokenizer.texts_to_sequences(abstracts)
caption_seqs = tokenizer.texts_to_sequences(captions)

# Pad sequences to ensure uniform length
max_len = 250  #i choosed 150 the average length of abstracts and captions
headline_seqs_padded = np.array(pad_sequences(headline_seqs, maxlen=max_len, padding='post'))
abstract_seqs_padded = np.array(pad_sequences(abstract_seqs, maxlen=max_len, padding='post'))
caption_seqs_padded = np.array(pad_sequences(caption_seqs, maxlen=max_len, padding='post'))

# Display tokenized and padded sequences
print("Tokenized and Padded Headline Sequences:")
print(headline_seqs_padded)
print("\nTokenized and Padded Abstract Sequences:")
print(abstract_seqs_padded)
print("\nTokenized and Padded Caption Sequences:")
print(caption_seqs_padded)


Tokenized and Padded Headline Sequences:
[[  121 26857     5 ...     0     0     0]
 [  262     7   142 ...     0     0     0]
 [ 4155  4506     6 ...     0     0     0]
 ...
 [    2   171  3232 ...     0     0     0]
 [ 1763    13   341 ...     0     0     0]
 [   12    42  1239 ...     0     0     0]]

Tokenized and Padded Abstract Sequences:
[[ 5410  5373    54 ...     0     0     0]
 [    1   264 25630 ...     0     0     0]
 [ 6508    17     1 ...     0     0     0]
 ...
 [    1   383 12631 ...     0     0     0]
 [    2   909   519 ...     0     0     0]
 [ 1277 49727 70093 ...     0     0     0]]

Tokenized and Padded Caption Sequences:
[[12138 12643     5 ...     0     0     0]
 [ 3922 29902     3 ...     0     0     0]
 [    1  4155  1406 ...     0     0     0]
 ...
 [ 4464   130 12631 ...     0     0     0]
 [   71  3811   587 ...     0     0     0]
 [ 1277 49727 90276 ...     0     0     0]]





































































































## B.Image dataset

The rar file in this folder are the only training images, I splitted them from th whole image folder and seperate them into a new folder called filtered images, because i am working on the google colab (cloud based), i had to upload this folder, for easy uploading i zipped the file and upload it to google drive, it is shown on the read me file on github more details about what are  the steps that I followed for the splitting into seperated folder.
Next cell, it is showing how unzipping will be done

In [ ]:

# Define the path to the RAR file in Google Drive
rar_file_path = '/content/drive/MyDrive/filtered_images.rar'

# Define the directory to extract the contents
extracted_dir_path = '/content/drive/MyDrive/training_caption'

# Create the directory if it doesn't exist
os.makedirs(extracted_dir_path, exist_ok=True)

# Extract the contents of the RAR file
with rarfile.RarFile(rar_file_path, 'r') as rar_ref:
    rar_ref.extractall(extracted_dir_path)

print("Extraction completed successfully.")


In [ ]:
extracted_dir_path = '/content/drive/MyDrive/training_cap/filtered_images'

# Count the number of files after extraction
after_extraction_count = len(os.listdir(extracted_dir_path))
print('Nmber of images after extraction from zip file ', after_extraction_count)

Nmber of images after extraction from zip file  48988


**Image resizing**
 I will choose the Resnet pretrained model, it is trained mostly on images has size 224*224, so i will resize all images to this size



In [ ]:

# Input directory
directory = '/content/drive/MyDrive/training_cap/filtered_images'

# Iterate through all files in the directory
for filename in os.listdir(directory):
    # Open the image file
    image_path = os.path.join(directory, filename)
    image = Image.open(image_path)

    # Convert RGBA image to RGB
    if image.mode == 'RGBA':
        image = image.convert('RGB')

    # Resize the image to 224x224 pixels
    resized_image = image.resize((224, 224))

    # Save the resized image
    resized_image.save(os.path.join(directory, "resized_" + filename))



**Image Normalization**

In [ ]:

def normalize_image(image):
    # Convert image to numpy array
    img_array = np.array(image)
    # Normalize pixel values to range [0, 1]
    normalized_img = img_array / 255.0
    return normalized_img

In [ ]:


directory = '/content/drive/MyDrive/training_cap/filtered_images'

# Iterate through all files in the directory
for filename in os.listdir(directory):
    # Open the resize image
    image_path = os.path.join(directory, filename)
    resized_image = Image.open(image_path)

    # Normalize the image
    normalized_image = normalize_image(resized_image)

    # Save the normalized image, replacing the original resized image
    np.save(image_path, normalized_image)


In [ ]:
directory = '/content/drive/MyDrive/training_cap/filtered_images'

# Iterate through all files in the directory
for filename in os.listdir(directory):
    # Open the  image
    image_path = os.path.join(directory, filename)
    _image = Image.open(image_path)

    # Save the  image
    np.save(image_path, _image)


**Feature Extraction using ResNet**

In [ ]:

def extract_resnet_features(directory):
    # Load pre-trained ResNet model exclude the top layers
    resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

    # Define the new directory to extract the contents
    extracted_dir_path = '/content/drive/MyDrive/training_cap/filtered_images'

    # Create the directory if it doesn't exist
    os.makedirs(extracted_dir_path, exist_ok=True)

    # Initialize an empty list to store extracted features
    features_list = []

    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        # Open the normalized image
        image_path = os.path.join(directory, filename)
        normalized_image = np.load(image_path)

        # Resize the image to match ResNet input size
        resized_image = Image.fromarray((normalized_image * 255).astype(np.uint8)).resize((224, 224))

        # Preprocess the image for ResNet model
        preprocessed_image = preprocess_input(np.array(resized_image))

        # Reshape the image to match ResNet input shape (batch size = 1)
        preprocessed_image = np.expand_dims(preprocessed_image, axis=0)

        # Extract features using ResNet model
        features = resnet_model.predict(preprocessed_image)

        # Append the extracted features to the list
        features_list.append(features)

    # Convert the list of features to a numpy array
    features_array = np.array(features_list)

    return features_array




In [ ]:

directory = '/content/drive/MyDrive/training_cap/filtered_images'

# Extract features from normalized images using ResNet
extracted_features = extract_resnet_features(directory)

# Save the extracted features for later use
np.save("resnet_features.npy", extracted_features)


ValueError: Cannot load file containing pickled data when allow_pickle=False

**Data Formatting:** Organize the extracted image features along with the corresponding textual data (headlines, abstracts) in a format suitable for training the captioning model. This typically involves creating pairs of image features and corresponding textual sequences.



# 3. Model Architecture

In [ ]:
# Define input layers
headline_input = Input(shape=(headline_max_len,), name='headline_input')
abstract_input = Input(shape=(abstract_max_len,), name='abstract_input')
image_features_input = Input(shape=(image_features_shape,), name='image_features_input')

# Text encoding layers
embedding_layer = Embedding(vocab_size, embedding_dim)
headline_embedding = embedding_layer(headline_input)
abstract_embedding = embedding_layer(abstract_input)

# LSTM encoding for text
lstm_units = 256
headline_lstm = LSTM(units=lstm_units, return_sequences=True)(headline_embedding)
abstract_lstm = LSTM(units=lstm_units, return_sequences=True)(abstract_embedding)

# Combine encoded representations
concatenated_features = Concatenate(axis=1)([headline_lstm, abstract_lstm, image_features_input])

# Attention mechanism
attention = Attention()
context_vector = attention([concatenated_features, concatenated_features])
attention_output = tf.keras.layers.Concatenate(axis=-1)([context_vector, concatenated_features])

# Decoder LSTM
decoder_lstm = LSTM(units=lstm_units, return_sequences=True)(attention_output)

# Output layer for caption generation
output_caption = Dense(vocab_size, activation='softmax')(decoder_lstm)

# Define the model
model = Model(inputs=[headline_input, abstract_input, image_features_input], outputs=output_caption)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Print the model summary
model.summary()


# 4. Model Training

In [ ]:
# Define training data
train_data = {
    'headline_input': headline_train,
    'abstract_input': abstract_train,
    'image_features_input': image_features_train
}
train_labels = captions_train

# Define validation data
val_data = {
    'headline_input': headline_val,
    'abstract_input': abstract_val,
    'image_features_input': image_features_val
}
val_labels = captions_val

# Train the model
epochs = 10
batch_size = 32

history = model.fit(
    train_data, train_labels,
    validation_data=(val_data, val_labels),
    epochs=epochs,
    batch_size=batch_size
)

# Plot training history
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


# 5. Model Evaluation

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# Generate captions using the trained model
generated_captions = ...  # Replace ... with the generated captions for the testing set
reference_captions = ...  # Replace ... with the reference captions for the testing set

# Convert reference captions to list of lists (each sublist contains reference captions for a single sample)
reference_captions = [[caption.split()] for caption in reference_captions]

# Convert generated captions to list of lists (each sublist contains a generated caption)
generated_captions = [caption.split() for caption in generated_captions]

# Calculate BLEU score
bleu_score = corpus_bleu(reference_captions, generated_captions)

print("BLEU Score:", bleu_score)


# Conclusion

In images, the idea was to resize and normalize images but due to limited space and disk,runtime crashed many times so I continued to work with images in their original form but i kept the line of codes to show that this is a good practice.